In [4]:
import os
import re
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
import ast
import json
from glob import glob

In [9]:
class_names_kor = ['bg','person', 'bike', 'lean']
class_names = ['bg','person', 'bike', 'lean']

folder_Tongyoung = '/home/mvlab/Downloads/dataset/통영/'
os.path.isdir(folder_pig)

True

In [28]:
folder_Tongyoung_target = folder_Tongyoung + '서영재/'
os.path.isdir(folder_Tongyoung_target)

True

In [29]:
def add_folder_to_file_name(folder_pig, path_csv):
    path_csv = folder_pig + path_csv
    path_weight = "retinanet/pig_efficient_det_BiFPN_D3"
    print(os.path.isdir(folder_pig), os.path.isfile(path_csv))
    if os.path.isfile(path_csv):
        
        df = pd.read_csv(path_csv)
        df['filename'] = folder_pig + df['filename']
        print('df', df.shape, df['filename'].nunique(), df['region_attributes'].nunique()  )
        print(df['region_attributes'].unique())
        return df
    else:
        return None

In [30]:
def read_df(folder_pig, path_csv):
    path_csv = folder_pig + path_csv
    if path_csv.endswith('xlsx'):
        df = pd.read_excel(path_csv)
    else:
        df = pd.read_csv(path_csv)
    
    print(df.shape)
    return df

In [31]:
df = read_df(folder_Tongyoung_target, 'via_project_27Jan2021_15h16m_csv.csv.xlsx')#21940, 24408

(482, 7)


In [32]:
#df = df6
#df = pd.concat((df1, df2, df3, df4, df5, df6), axis=0)
df.shape

(482, 7)

In [33]:
folder_Tongyoung_target

'/home/mvlab/Downloads/dataset/통영/서영재/'

In [34]:
#path_images = glob(folder_pig + 'pig1-2020-11/*/*/*.jpg')
path_images = glob(folder_Tongyoung_target + '*.jpg')
len(path_images), path_images[0]

(481, '/home/mvlab/Downloads/dataset/통영/서영재/16620.jpg')

In [35]:
dict_full_path = dict()
for full_path in path_images:
    cut_path = full_path.split(os.sep)[-1]
    dict_full_path[cut_path] = full_path
    #print(cut_path)
len(dict_full_path)

481

In [36]:
df['region_attributes'].unique()

array(['{}', '{"class":"lean"}'], dtype=object)

In [47]:
df['region_id'].unique()

array([0, 1])

In [70]:
df

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,00000.jpg,715145,{},0,0,{},{}
1,00060.jpg,713640,{},0,0,{},{}
2,00120.jpg,685648,{},0,0,{},{}
3,00180.jpg,698387,{},0,0,{},{}
4,00240.jpg,707386,{},0,0,{},{}
...,...,...,...,...,...,...,...
477,28560.jpg,706714,{},0,0,{},{}
478,28620.jpg,688179,{},0,0,{},{}
479,28680.jpg,687241,{},0,0,{},{}
480,28740.jpg,688895,{},0,0,{},{}


In [84]:
def convert_VIA_to_DarkLabel(df):
    df_clone = df.copy()
    df_clone['cls'] = 0
    df_clone['id'] = -1
    df_clone['x'] = 0
    df_clone['y'] = 0
    df_clone['w'] = 0
    df_clone['h'] = 0
    for i in range(len(df)):
    
        sample_row = df.iloc[i]
        filename = sample_row['filename']    
        region_id = sample_row['region_id']
        
        region_count = sample_row['region_count']
        box = sample_row['region_shape_attributes']    
        region_attributes = sample_row['region_attributes'] 
        
        if '(' in filename:
            print('()', filename)
            
        if int(region_count) > 0:
            
            try:
                box_map = ast.literal_eval(box)    
                region_attributes = ast.literal_eval(region_attributes)        
                cls = region_attributes['class']
                
                x0 = box_map['x']
                y0 = box_map['y']
                width = box_map['width']
                height = box_map['height']  
                
                #cls_name = class_names[int(cls)]
                #print(i)
                df_clone.loc[i,'cls'] = cls#_name
                #df_clone.iloc[i]['id'] = -1
                df_clone.loc[i,'x'] = x0
                df_clone.loc[i,'y'] = y0
                df_clone.loc[i,'w'] = width
                df_clone.loc[i,'h'] = height

            except:
                print('except filename',i, filename)
                print('box', box)
                print('region_attributes', region_attributes)
                continue

            if len(box_map.keys())<1 or len(region_attributes.keys())<1 :
                #print(i, 'no_keys_skip', filename.split(os.sep)[-1])
                continue
    
    df_out = df_clone[['filename', 'cls', 'id', 'x', 'y', 'w', 'h']]
    df_out = df_out[df_out['w']>0]
    return df_out

In [85]:
df_darklabel =  convert_VIA_to_DarkLabel(df)
df_darklabel#.columns

,filename,cls,id,x,y,w,h
42,02520.jpg,lean,-1,899,567,31,110
43,02580.jpg,lean,-1,894,566,38,106
44,02640.jpg,lean,-1,894,566,38,106
45,02700.jpg,lean,-1,894,566,38,106
46,02760.jpg,lean,-1,894,566,38,106
...,...,...,...,...,...,...,...
464,27780.jpg,lean,-1,896,526,30,62
465,27840.jpg,lean,-1,896,526,30,62
466,27900.jpg,lean,-1,896,526,30,62
467,27960.jpg,lean,-1,896,526,30,62


In [86]:
df_darklabel.to_csv('image_label.txt', header=None, index=None)

In [39]:
annotation = dict()
object_width_list = []
for i in range(len(df)):
    
    sample_row = df.iloc[i]
    filename = sample_row['filename']    
    region_id = sample_row['region_id']
    box = sample_row['region_shape_attributes']    
    region_attributes = sample_row['region_attributes'] 
    if '(' in filename:
        print('()', filename)
    try:
        box_map = ast.literal_eval(box)    
        region_attributes = ast.literal_eval(region_attributes)        
    except:
        print('except filename',i, filename)
        print('box', box)
        print('region_attributes', region_attributes)
        continue
    
    if len(box_map.keys())<1 or len(region_attributes.keys())<1 :
        #print(i, 'no_keys_skip', filename.split(os.sep)[-1])
        continue
        
    cls = region_attributes['class']
    if cls in class_names_kor:
        cls_num = class_names_kor.index(cls)
    elif cls in class_names:
        cls_num = class_names.index(cls)
    else:
        print('cls unknown', cls)
        print('filename', filename)
        continue
        #xit()
        
    x0 = box_map['x']
    y0 = box_map['y']
    width = box_map['width']
    height = box_map['height']    
   
    object_width_list.append(width)    
    bbox = [cls_num, x0, y0, x0 + width, y0 + height]
    
    filename = dict_full_path[filename]
    if filename not in annotation.keys():
        annotation[filename] = []
    
    annotation[filename].append(bbox)            
    #print(i, df.loc[i, 'filename'], width, height)

In [40]:
len(annotation)

316

In [41]:
def load_pig():
    input_list = []
    bbox_list = []
    path_list = []
    for path_image in annotation:
        cls_bbox = annotation[path_image]
        cls_bbox = np.array(cls_bbox).reshape([-1, 5])
        cls = cls_bbox[:, 0:1]
        bbox = np.array(cls_bbox[:, 1:])
        
        #print('path_image', path_image)
        if True:
            for full_path in path_images:
                if path_image in full_path:
                    path_image = full_path

        if os.path.isfile(path_image):
            img = Image.open(path_image)    
            #img = img.resize((1000,500))
           
            scale = np.array((img.width, img.height, img.width, img.height))
            scale = np.reshape(scale, (1, 4))       
            #print(key, cls, cls_bbox.dtype, cls_bbox, 'wh',img.width, img.height)
            img_arr = np.array(img)        
            del img
            bbox_norm = bbox.astype(np.float) / scale.astype(np.float)
            cls = np.array(cls).reshape((-1, 1))
            cls_bbox_norm = np.concatenate((cls, bbox_norm), axis=1)
                
            input_list.append(img_arr)
            bbox_list.append(cls_bbox_norm)
            path_list.append(full_path)
            if len(input_list)%100==0:        
                print(len(annotation), len(input_list), len(bbox_list))   
        else:
            print('not exist', path_image)

    print(len(input_list), len(bbox_list))
    return input_list, bbox_list
    
input_list_pig, bbox_list_pig = load_pig()#1709, 1759

316 100 100
316 200 200
316 300 300
316 316


In [42]:
boxes = np.concatenate(bbox_list_pig, 0)
boxes.shape, np.sum(boxes[:, 0]>3)

((317, 5), 0)

In [43]:
len(input_list_pig)

316

In [44]:
def crop_bbox_image(image, boxes):
    img_objects = []
    image = np.array(image)
    for box in boxes:        
        box = box.astype(np.int)
        x1, y1, x2, y2 = box
        w = x2 - x1
        h = y2 - y1        
        crop_image_arr = image[y1:y2, x1:x2]
        ch, cw, cc = crop_image_arr.shape
        if ch>1 and cw>1:
            img_objects.append(crop_image_arr)
        else:
            print('crop_bbox_image', x2-x1, y2-y1, 'crop_image_arr.shape', crop_image_arr.shape)
        
    return img_objects
    
    
def attach_crop_image(image, boxes, max_crop=200):
        
    crop_bbox_arr = crop_bbox_image(image, np.array(boxes)[:max_crop])
    bbox_k = len(crop_bbox_arr)
    max_col = 30
    
    if bbox_k > 0:
        img_h, img_w, img_c = image.shape
        object_img_w = img_w//bbox_k        
        resize_h = img_h // 8
        resize_w = img_w // bbox_k  
        resize_w = min(max(resize_w, img_w//max_col), img_w//8)
        
        footer_h = resize_h * (1 + (bbox_k-1)//max_col)
        footer = np.zeros((footer_h, img_w, img_c), np.uint8)
        
        for i in range(min(bbox_k, max_crop)):
            crop_arr = crop_bbox_arr[i]
            crop_img = Image.fromarray(crop_arr)                
            crop_img = crop_img.resize((resize_w, resize_h))
            crop_arr_resized = np.array(crop_img)
            offset_y = (i//max_col) * resize_h
            offset_x = (i%max_col) * resize_w
            footer[offset_y:offset_y+resize_h, offset_x:offset_x+resize_w] = crop_arr_resized

        seperate_line = np.zeros_like(footer[:2])
        image = np.concatenate((image, seperate_line, footer), axis=0)    
    return image    

In [45]:
def visualize_detections_simple(
    image, boxes, classes, figsize=(12, 12), linewidth=1, color=[0, 0, 1]
):
    """Visualize Detections"""
    image = np.array(image, dtype=np.uint8)    
    
    img_h, img_w, img_c = image.shape
    
    image = attach_crop_image(image, boxes, max_crop=100)
    
    plt.figure(figsize=figsize)
    plt.axis("off")
    plt.imshow(image)
    ax = plt.gca()
    boxes_width = boxes[:, 2] - boxes[:, 0]
    boxes_height = boxes[:, 3] - boxes[:, 1]
    box_min_width = np.min(boxes_width)
    box_max_width = np.max(boxes_width)
    title = str.format('(%dx%d) %d box, width:%d ~ %d' 
                       %(img_h, img_w, len(boxes), box_min_width, box_max_width))
    plt.title(title)
    for box, cls in zip(boxes, classes):
        x1, y1, x2, y2 = box        
        w, h = x2 - x1, y2 - y1
        color = [1,1,1]
        patch = plt.Rectangle(
            [x1, y1], w, h, fill=False, edgecolor=color, linewidth=linewidth
        )
        ax.add_patch(patch)
        if len(boxes) < 70:
            score_txt = class_names[int(cls)]
            ax.text(x1, y1, score_txt, bbox={"facecolor": [1,1,0], "alpha": 0.4}, clip_box=ax.clipbox, clip_on=True,)
        
    plt.show()
    return ax

def display_data(X, BBOX, stride=1):
    for i in range(len(X)):
        if i%stride==0:
            img_arr = X[i]
            sample_box = BBOX[i]
            label = sample_box[:, 0]
            bbox = sample_box[:, 1:]

            h, w, c = img_arr.shape
            scale = np.array((w, h, w, h))
            scale = np.reshape(scale, (1, 4))
            bbox_norm = bbox.astype(np.float) * scale.astype(np.float)
            #print('bbox_norm', bbox, bbox_norm)
            print(i, np.unique(label))
            ax = visualize_detections_simple(img_arr,bbox_norm,label)

s = len(input_list_pig)*0
e = -1
display_data(input_list_pig[s:e], bbox_list_pig[s:e], stride=10)